<a href="https://colab.research.google.com/github/marceloamoraes/githubtest/blob/main/Project_Travel_Package_Purchase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Import necessary libraries**

In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
import scipy.stats as stats
from sklearn import metrics
from sklearn import tree
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')


### **Read the dataset**

In [2]:
df = pd.read_excel('Tourism.xlsx', sheet_name='Tourism')

In [3]:
df.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisited,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisited,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Single,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Single,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,NaN,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0


In [4]:
df.tail()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisited,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisited,Designation,MonthlyIncome
4883,204883,1,49.0,Self Enquiry,3,9.0,Small Business,Male,3,5.0,Deluxe,4.0,Unmarried,2.0,1,1,1,1.0,Manager,26576.0
4884,204884,1,28.0,Company Invited,1,31.0,Salaried,Male,4,5.0,Basic,3.0,Single,3.0,1,3,1,2.0,Executive,21212.0
4885,204885,1,52.0,Self Enquiry,3,17.0,Salaried,Female,4,4.0,Standard,4.0,Married,7.0,0,1,1,3.0,Senior Manager,31820.0
4886,204886,1,19.0,Self Enquiry,3,16.0,Small Business,Male,3,4.0,Basic,3.0,Single,3.0,0,5,0,2.0,Executive,20289.0
4887,204887,1,36.0,Self Enquiry,1,14.0,Salaried,Male,4,4.0,Basic,4.0,Unmarried,3.0,1,3,1,2.0,Executive,24041.0


### **Getting some information and data types about the data¶**

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4888 entries, 0 to 4887
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   CustomerID               4888 non-null   int64  
 1   ProdTaken                4888 non-null   int64  
 2   Age                      4662 non-null   float64
 3   TypeofContact            4863 non-null   object 
 4   CityTier                 4888 non-null   int64  
 5   DurationOfPitch          4637 non-null   float64
 6   Occupation               4888 non-null   object 
 7   Gender                   4888 non-null   object 
 8   NumberOfPersonVisited    4888 non-null   int64  
 9   NumberOfFollowups        4843 non-null   float64
 10  ProductPitched           4888 non-null   object 
 11  PreferredPropertyStar    4862 non-null   float64
 12  MaritalStatus            4888 non-null   object 
 13  NumberOfTrips            4748 non-null   float64
 14  Passport                

**Observations**
* There are several variables with missing values: Age, TypeofContact, DurationOfPitch, NumberOfFollowups, PreferredPropertyStar, NumberOfTrips, NumberOfChildrenVisited and MonthlyIncome
* All 'Object' variable should be changed to 'Categorical' since they represent a sort of category.
* At this point feature engineering does not seem to be required.

In [6]:
df.columns

Index(['CustomerID', 'ProdTaken', 'Age', 'TypeofContact', 'CityTier',
       'DurationOfPitch', 'Occupation', 'Gender', 'NumberOfPersonVisited',
       'NumberOfFollowups', 'ProductPitched', 'PreferredPropertyStar',
       'MaritalStatus', 'NumberOfTrips', 'Passport', 'PitchSatisfactionScore',
       'OwnCar', 'NumberOfChildrenVisited', 'Designation', 'MonthlyIncome'],
      dtype='object')

In [7]:
#Fixing the data type
df["TypeofContact"] = df["TypeofContact"].astype("category")
df["Occupation"] = df["Occupation"].astype("category")
df["Gender"] = df["Gender"].astype("category")
df["ProductPitched"] = df["ProductPitched"].astype("category")
df["MaritalStatus"] = df["MaritalStatus"].astype("category")
df["Designation"] = df["Designation"].astype("category")

In [8]:
#Searching for duplicated values
duplicated = df.duplicated()
sum(duplicated)

0

In [9]:
df.describe().T  # quick summary of numeric features

,count,mean,std,min,25%,50%,75%,max
CustomerID,4888.0,202443.500000,1411.188388,200000.0,201221.75,202443.5,203665.25,204887.0
ProdTaken,4888.0,0.188216,0.390925,0.0,0.00,0.0,0.00,1.0
Age,4662.0,37.622265,9.316387,18.0,31.00,36.0,44.00,61.0
CityTier,4888.0,1.654255,0.916583,1.0,1.00,1.0,3.00,3.0
DurationOfPitch,4637.0,15.490835,8.519643,5.0,9.00,13.0,20.00,127.0
NumberOfPersonVisited,4888.0,2.905074,0.724891,1.0,2.00,3.0,3.00,5.0
NumberOfFollowups,4843.0,3.708445,1.002509,1.0,3.00,4.0,4.00,6.0
PreferredPropertyStar,4862.0,3.581037,0.798009,3.0,3.00,3.0,4.00,5.0
NumberOfTrips,4748.0,3.236521,1.849019,1.0,2.00,3.0,4.00,22.0
Passport,4888.0,0.290917,0.454232,0.0,0.00,0.0,1.00,1.0


**Observations**
* 

### **Missing Values**

In [10]:
df.isnull().sum() # lots of columns don't have missingness

CustomerID                   0
ProdTaken                    0
Age                        226
TypeofContact               25
CityTier                     0
DurationOfPitch            251
Occupation                   0
Gender                       0
NumberOfPersonVisited        0
NumberOfFollowups           45
ProductPitched               0
PreferredPropertyStar       26
MaritalStatus                0
NumberOfTrips              140
Passport                     0
PitchSatisfactionScore       0
OwnCar                       0
NumberOfChildrenVisited     66
Designation                  0
MonthlyIncome              233
dtype: int64

In [11]:
# most rows don't have missing values now
num_missing = df.isnull().sum(axis=1)
num_missing.value_counts()

0    4128
1     533
2     202
3      25
dtype: int64

In [12]:
# checking patterns in the missingness
df[num_missing == 2].sample(n=5)

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisited,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisited,Designation,MonthlyIncome
1120,201120,0,NaN,Self Enquiry,3,22.0,Salaried,Female,2,3.0,Deluxe,4.0,Single,3.0,0,1,0,0.0,Manager,NaN
184,200184,0,53.0,Self Enquiry,1,NaN,Small Business,Female,2,2.0,Deluxe,5.0,Married,2.0,0,2,1,1.0,Manager,NaN
518,200518,0,NaN,Self Enquiry,3,13.0,Small Business,Female,2,4.0,Deluxe,3.0,Single,1.0,0,2,1,1.0,Manager,NaN
2272,202272,0,33.0,Company Invited,3,NaN,Salaried,Female,1,3.0,Deluxe,4.0,Married,1.0,0,4,0,0.0,Manager,NaN
581,200581,0,NaN,Self Enquiry,1,6.0,Salaried,Male,2,5.0,Basic,3.0,Divorced,4.0,0,4,1,0.0,Executive,NaN


- Based on some research, if the missingness is below 5% of the dataset, dropping the data will not significantly affect results.

In [13]:
# Dropping rows with 2 or 3 missing values
index = df.loc[num_missing > 1].index.tolist()
df.drop(df.index[[index]], inplace=True)

In [18]:
# rechecking missing values
num_missing = df.isnull().sum(axis=1)
num_missing.value_counts()

0    4128
1     533
dtype: int64

In [22]:
df.isnull().sum() # rechecking where are the missing values

CustomerID                   0
ProdTaken                    0
Age                         96
TypeofContact                0
CityTier                     0
DurationOfPitch            154
Occupation                   0
Gender                       0
NumberOfPersonVisited        0
NumberOfFollowups           45
ProductPitched               0
PreferredPropertyStar       26
MaritalStatus                0
NumberOfTrips              140
Passport                     0
PitchSatisfactionScore       0
OwnCar                       0
NumberOfChildrenVisited     66
Designation                  0
MonthlyIncome                6
dtype: int64

In [24]:
#continuation of filling missing values 
df['Age'].fillna(df['Age'].median(), inplace=True)
df['DurationOfPitch'].fillna(df['DurationOfPitch'].median(), inplace=True)
df['NumberOfFollowups'].fillna(df['NumberOfFollowups'].median(), inplace=True)
df['PreferredPropertyStar'].fillna(df['PreferredPropertyStar'].median(), inplace=True) 
df['NumberOfTrips'].fillna(df['NumberOfTrips'].median(), inplace=True) 
df['NumberOfChildrenVisited'].fillna(df['NumberOfChildrenVisited'].median(), inplace=True) 
df['MonthlyIncome'].fillna(df['MonthlyIncome'].median(), inplace=True) 

In [26]:
df.isnull().sum() # rechecking if it still has missing values

CustomerID                 0
ProdTaken                  0
Age                        0
TypeofContact              0
CityTier                   0
DurationOfPitch            0
Occupation                 0
Gender                     0
NumberOfPersonVisited      0
NumberOfFollowups          0
ProductPitched             0
PreferredPropertyStar      0
MaritalStatus              0
NumberOfTrips              0
Passport                   0
PitchSatisfactionScore     0
OwnCar                     0
NumberOfChildrenVisited    0
Designation                0
MonthlyIncome              0
dtype: int64

In [27]:
# looking at value counts for non-numeric features

num_to_display = 10  # defining this up here so it's easy to change later if I want
for colname in df.dtypes[df.dtypes == 'category'].index:
    val_counts = df[colname].value_counts(dropna=False)  # i want to see NA counts
    print(val_counts[:num_to_display])
    if len(val_counts) > num_to_display:
        print(f'Only displaying first {num_to_display} of {len(val_counts)} values.')
    print('\n\n') # just for more space between 

Self Enquiry       3311
Company Invited    1350
Name: TypeofContact, dtype: int64



Salaried          2267
Small Business    1972
Large Business     420
Free Lancer          2
Name: Occupation, dtype: int64



Male       2785
Female     1721
Fe Male     155
Name: Gender, dtype: int64



Basic           1757
Deluxe          1590
Standard         742
Super Deluxe     342
King             230
Name: ProductPitched, dtype: int64



Married      2231
Divorced      903
Single        845
Unmarried     682
Name: MaritalStatus, dtype: int64



Executive         1757
Manager           1590
Senior Manager     742
AVP                342
VP                 230
Name: Designation, dtype: int64





In [28]:
#Correting some values with typo in the Genre
df['Gender'] = df['Gender'].replace('Fe Male', 'Female')